## Onboard Wind Hazard Maps from European Central Bank (ECB) to OS-C S3 bucket

The data was directly sendt by the ECB via email and covers wind speed return periods for Europe. 

Return periods: 5, 10, 50, 100, and 500 years.

The data is provided for every european nuts_id, so we need to download [eurostat geojson](https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts) to convert to lat/lon.

## Create Zarr from shape and Affine transformation

<span style="color:blue">Note: this file must be located in /hazard/src/ for the dependencies to work</span>

In [1]:
import sys
import os
import s3fs
import zarr
import numpy as np
import rasterio
import math
import xarray as xr
import math
import pyproj
import pandas as pd
import geopandas as gpd

from pyproj.crs import CRS
from affine import Affine

from hazard.sources.osc_zarr import OscZarr

c:\ProgramData\Anaconda3\lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
# https://console-openshift-console.apps.odh-cl1.apps.os-climate.org/k8s/ns/sandbox/secrets/physrisk-dev-s3-keys
# Hazard indicators bucket
default_staging_bucket = 'physrisk-hazard-indicators-dev01'
prefix = 'hazard'

# Acess key and secret key are stored as env vars OSC_S3_HI_ACCESS_KEY and OSC_S3_HI_SECRET_KEY, resp.
s3 = s3fs.S3FileSystem(anon=False, key=os.environ["OSC_S3_HIdev01_ACCESS_KEY"], secret=os.environ["OSC_S3_HIdev01_SECRET_KEY"])

# Define zarr group
zarr_storage = 'hazard.zarr'
group_path = os.path.join(default_staging_bucket, prefix, zarr_storage).replace('\\','/')
store = s3fs.S3Map(root=group_path, s3=s3, check=False)
root = zarr.group(store=store, overwrite=False) 

# zarr_ storage tree
root.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='physrisk-hazard-indicators-dev01', …

In [3]:
# List folder files
s3.ls(os.path.join(default_staging_bucket, prefix).replace('\\','/'))

['physrisk-hazard-indicators-dev01/hazard/hazard.zarr',
 'physrisk-hazard-indicators-dev01/hazard/riverflood_JRC_RP_hist.zarr']

In [4]:
# Create OscZarr object to interact with the bucket.
oscZ = OscZarr(bucket=default_staging_bucket,
        prefix=prefix,
        s3=s3,
        store=store)

In [5]:
# Path to the nc file. 

base_path_hazard = os.path.join(os.getenv("physical_risk_database"), 'hazard')

hazard_type = 'Wind'
datasource = 'ECB'

inputfile_path = os.path.join(base_path_hazard, hazard_type, datasource)
data_filename = 'wind_gumbel_ECB.csv'
nuts_id_filename = 'NUTS_LB_2021_4326.geojson'

# There is one .nc file with 8 return periods
inputfile = os.path.join(inputfile_path, data_filename)
ws = pd.read_csv(inputfile)
ws.head(5)

,rp,nuts_id,gustspeed,ctr
0,5,AL011,27.440091,AL
1,5,AL012,37.324202,AL
2,5,AL013,27.117959,AL
3,5,AL014,33.951803,AL
4,5,AL015,36.102142,AL
...,...,...,...,...
7430,500,UKN0C,64.945687,UK
7431,500,UKN0D,45.752930,UK
7432,500,UKN0E,45.135838,UK
7433,500,UKN0F,53.088222,UK


In [6]:
# Load nuts_id lat-lon coordinates from eurostat (see link below)
# https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts
inputfile = os.path.join(inputfile_path, nuts_id_filename)
nuts = gpd.read_file(inputfile)
nuts = nuts[['NUTS_ID', 'geometry']]
nuts.head(5)

,NUTS_ID,geometry
0,DEF0,POINT (9.82120 54.08040)
1,DEG0,POINT (11.02180 50.90440)
2,DK01,POINT (12.32960 55.86180)
3,DK02,POINT (11.75090 55.43760)
4,DK03,POINT (9.05690 55.40380)
...,...,...
2005,NO071,POINT (14.78250 66.69440)
2006,NO091,POINT (8.70540 59.49660)
2007,NO,POINT (9.29810 61.38750)
2008,NO02,POINT (10.42600 61.44500)


In [51]:
# Create longitude and latitude vector. Create wind gust speed matrix.
return_periods = [5, 10, 50, 100, 500]
ws_rp_matrix = np.zeros(ws.shape[0])

longitude = []
latitude = []
for i in range(ws.shape[0]):
    ws_ = ws.gustspeed[i]
    rp = ws.rp[i]
    rp_index = return_periods.index(rp)
    nuts_id = ws.nuts_id[i]
    lon = nuts[nuts.NUTS_ID == nuts_id].geometry.x
    lat = nuts[nuts.NUTS_ID == nuts_id].geometry.y

    longitude.append(lon.values[0])
    latitude.append(lat.values[0])

    ws_rp_matrix[i] = ws_

ws_rp_matrix = ws_rp_matrix.reshape(len(return_periods), (int(ws.shape[0] / len(return_periods))))


In [71]:
# As you can notice the file privdes data as a vector
# We must create a grid

# Create latitude and longitude grid
min_lat, max_lat = min(latitude), max(latitude)
min_lon, max_lon = min(longitude), min(longitude)

total_size = ws_rp_matrix.shape[1]
small_size = total_size
grid = np.meshgrid(np.linspace(min_lon, max_lon, total_size), np.linspace(min_lat, max_lat, small_size))

# Create and empty matrix with zeros
ws_matrix = np.zeros((small_size, total_size, len(return_periods)))

In [73]:
# Save the data 
ws_matrix_name = os.path.join(inputfile_path, "ws_matrix.npy")

if "ws_matrix.npy" in os.listdir(inputfile_path):
    # Find the nearest point and and the ssl value

    for pos_i in range(total_size):
        lon_i = longitude[pos_i]
        lat_i = latitude[pos_i]
        ws_i = ws_rp_matrix[:, pos_i]
        
        aux_min = 500000
        for i in range(small_size):
            for j in range(total_size):
                lon_ij = grid[0][i, j]
                lat_ij = grid[1][i, j]

                dist = math.dist((lon_ij, lat_ij), (lon_i, lat_i))

                if dist < aux_min:
                    aux_min = dist
                    aux_min_i = (i, j)
        
        ws_matrix[aux_min_i[0], aux_min_i[1], :] = ws_i
        print(pos_i)

    ws_matrix = np.load(ws_matrix_name)
else:
    np.save(ws_matrix_name, ws_matrix)   

In [74]:
# Define zarr shape and coordinate system
width = ws_matrix.shape[1]
height = ws_matrix.shape[0]
shape = (height, width)
crs = str(CRS.from_epsg(3035))

In [75]:
# Transform from lat-lon coordinates to EPSD:3035 coordinates
proj = pyproj.Transformer.from_crs(4326, 3035, always_xy=True, authority='EPSG')
longitudes = grid[0]
latitudes = grid[1]
xs, ys = proj.transform(np.array(longitudes),  np.array(latitudes))

In [76]:
# Create Affine transformation
min_xs = xs.min()
max_xs = xs.max()
min_ys = ys.min()
max_ys = ys.max()

bounds = (min_xs, min_ys, max_xs, max_ys)
size = shape

# Compute the parameters of the georeference
A = (bounds[2] - bounds[0]) / size[0] # pixel size in the x-direction in map units/pixel
B = 0 # rotation about y-axis
C = 0 # rotation about x-axis
D = -(bounds[3] - bounds[1]) / size[1] # pixel size in the y-direction in map units, almost always negative
E = bounds[0] # x-coordinate of the center of the upper left pixel
F = bounds[3] # y-coordinate of the center of the upper left pixel

transform = Affine(A, B, C, D, E, F)
transform

Affine(1150.0541884724473, 0.0, 0.0,
       -2466.3573575621162, 1500541.8858596976, 5587894.087333741)

In [77]:
# Create data file inside zarr group with name dataset_name

# Name standard is: hazard_type + _ + hazard_subtype (if exists) + '_' + hist or scenario + '_' RP (return period) or event/ emulated + '_' + data_provider
dataset_name = 'wind_hist_RP_ECB'
group_path_array = os.path.join(group_path, dataset_name)
oscZ._zarr_create(path=group_path_array,
                  shape = shape,
                  transform = transform,
                  crs = str(crs),
                  overwrite=False,
                  return_periods=return_periods)

<zarr.core.Array '/physrisk-hazard-indicators-dev01/hazard/hazard.zarr/wind_hist_RP_ECB' (5, 1487, 1487) float32>

In [78]:
z = oscZ.root[group_path_array]
z.info

Name,/physrisk-hazard-indicators-dev01/hazard/hazard.zarr/wind_hist_RP_ECB
Type,zarr.core.Array
Data type,float32
Shape,"(5, 1487, 1487)"
Chunk shape,"(5, 1000, 1000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.FSStore
No. bytes,44223380 (42.2M)
No. bytes stored,734


## Steps to populate hazard.zarr/wind_hist_RP_ECB

### Step 2: Populate the raster file for every return period

In [79]:
chunck_size = 1000

for rt_i in range(len(return_periods)):
    for height_pos in range(0, height, chunck_size):
        for width_pos in range(0, width, chunck_size):

            z[rt_i,height_pos:height_pos+chunck_size, width_pos:width_pos+chunck_size] = ws_matrix[height_pos:height_pos+chunck_size, width_pos:width_pos+chunck_size, rt_i]